In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password2"

try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    session = driver.session()
    result = session.run("RETURN 1")
    value = result.single()[0]
    print("Connection successful. Returned value:", value)
except Exception as e:
    print("Connection error:", str(e))

Connection successful. Returned value: 1


In [48]:
# query = """
# MATCH (r:Recipe)
# WHERE r.course = 'One Pot Dish'
# SET r.course = 'Main Course'
# """

# with driver.session() as session:
#     with session.begin_transaction() as tx:
#         tx.run(query)


In [2]:
#function for retrieving recipes with all ingredients
def get_recipes_with_all_ingredients(ingredients_list, course, diet):
    query = """
    MATCH (r:Recipe)
    WHERE ALL(ingredientName IN $ingredients_list
              WHERE (:Ingredient {name: ingredientName})-[:Ingredient_In]->(r))
    AND r.course = $course AND r.diet = $diet
    RETURN r.name
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list, course=course, diet=diet)
            return [record["r.name"] for record in result]

In [3]:
#function to get weight of each ingredient  in the ingredient list
def get_ingredient_weight(ingredients_list):
    query = """
    MATCH (i:Ingredient)
    WHERE i.name IN $ingredients_list
    RETURN i.name, i.weight
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list)
            ingre_with_weight = {}
            for record in result:
                ingre_with_weight[record["i.name"]] = record["i.weight"]
            return ingre_with_weight

In [4]:
#function to get rating of a recipe
def get_recipe_rating(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.rating
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]

In [5]:
#function to get URL of recipe
def get_recipe_url(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.url
    """

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]
        

In [6]:
#function to get all ingredients in a recipe
def get_ingredients_in_recipe(recipe_name):
    query = """
    MATCH (i:Ingredient)-[:Ingredient_In]->(r:Recipe {name: $recipe_name})
    RETURN i.name
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return [record["i.name"] for record in result]

In [7]:
#Function to write cypher query to total weight of ingredients in the recipe
def get_total_weight_of_ingredients(ingredients_list):
    query = """
    MATCH (i:Ingredient)
    WHERE i.name IN $ingredients_list
    RETURN sum(i.weight)
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list)
            return result.single()[0]

In [34]:
#Function to write cypher query to get cook_time of recipe
def get_cook_time_of_recipe(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.cooktime
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]
        

In [9]:
#get nutrients and servings of recipe 
def get_nutrients(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.nutrients
    """

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]

In [30]:
#Function to get dishspecific ingredients
def get_dish_specific_ingredients(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.dish_specific
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]

In [35]:
#Function to get served_with recipes for a recipe
def get_served_with(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})-[:Served_With]->(r2:Recipe)
    RETURN r2.name
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return [record["r2.name"] for record in result]

In [43]:
#get the top 5 recipe which have more incoming edges in given list of recipes
def get_top_5_recipes(recipes_list):
    query = """
    MATCH (r1:Recipe)-[i:Served_With]->(r2:Recipe)
    WHERE r2.name IN $recipes_list
    RETURN r2.name, count(i) as count
    ORDER BY count DESC
    LIMIT 5
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipes_list=recipes_list)
            return [record["r2.name"] for record in result]

In [10]:
#function to extract until alphabet
def extract_until_alphabet(s):
    extracted = ""
    for char in s:
        if char.isalpha():
            break
        extracted += char
    return float(extracted)

In [11]:
weight_gain_weights = {
    'calories': 0.25,
    'total_fat': 0.15,
    'saturated_fat': 0.1,
    'fiber': 0.05,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.05,
}
weight_loss_weights = {
    'calories': 0.1,
    'total_fat': 0.1,
    'saturated_fat': 0.05,
    'fiber': 0.25,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.1,
}
balanced_diet_weights = {
    'calories': 0.15,
    'total_fat': 0.15,
    'saturated_fat': 0.1,
    'fiber': 0.15,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.05,
}
import ast
# calculate the gain weight max score from all the recipes
maxb=1.5142857142857142
maxg=2.378571428571429
def weight_gain(nutrients):
    score=0.0
    # print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in weight_gain_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        score=score/maxg
        return score
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass
    
def weight_loss(nutrients):
    
    score=0.0
# print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in weight_loss_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        return score
        # print(score)
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass
def balanced_diet(nutrients):
   
    score=0.0
# print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in balanced_diet_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        score=score/maxb
        return score
        # print(score)
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass

In [12]:
#function to stem the list of words
from nltk.stem import PorterStemmer
def stem_list(list1):
    ps = PorterStemmer()
    stemmed_list = []
    for i in list1:
        stemmed_list.append(ps.stem(i))
    return stemmed_list



In [13]:
import itertools
#combinations of a list elements with random 75% of elements of original list 
def get_combinations_75(list1):
    list1_75_combinations = []
    for i in range(int(len(list1)*0.75), len(list1)):
        list1_75_combinations.extend(list(itertools.combinations(list1, i)))
    return list1_75_combinations

#combinations of a list elements with random 50% of elements of original list
def get_combinations_50(list1):
    # list1_50 = list1[:int(len(list1)*0.5)]
    list1_50_combinations = []
    for i in range(int(len(list1)*0.5) , int(len(list1)*0.75)):
        list1_50_combinations.extend(list(itertools.combinations(list1, i)))
    return list1_50_combinations

In [14]:
def get_recipes_with_ingredients_combinations_v3(ingredients_list, course, diet):
    total_recipe_list = []
    total_recipe_dict = dict()
    result1 = get_recipes_with_all_ingredients(ingredients_list, course, diet)
    total_recipe_list.extend(result1)
    for recipe in result1:
        total_recipe_dict[recipe] = 0.5

    combinations_75 = get_combinations_75(ingredients_list)
    for combination in combinations_75:
        result2 = get_recipes_with_all_ingredients(combination, course, diet)
        for recipe in result2:
            if recipe not in total_recipe_dict:
                ingre = get_ingredients_in_recipe(recipe)
                final_idf = get_total_weight_of_ingredients(ingre)/len(ingre)
                total_recipe_dict[recipe] = 0.375 * final_idf
                total_recipe_list.append(recipe)

    combinations_50 = get_combinations_50(ingredients_list)
    for combination in combinations_50:
        result3 = get_recipes_with_all_ingredients(combination, course, diet)
        for recipe in result3:
            if recipe not in total_recipe_dict:
                ingre = get_ingredients_in_recipe(recipe)
                final_idf = get_total_weight_of_ingredients(ingre)/len(ingre)
                total_recipe_dict[recipe] = 0.25 * final_idf
                total_recipe_list.append(recipe)

    return total_recipe_list, total_recipe_dict

In [26]:
ingredient_list = ['garam masala powd', 'paneer','fresh cream', 'butter', 'cashew nut', 'gram pan', 'cumin powd', 'kasuri methi']
course = "Main Course"
diet = "Vegetarian"

retreived_recipes_list, retreived_recipes_dict  = get_recipes_with_ingredients_combinations_v3(ingredient_list, course, diet)
#sort retrived_recipes_dict in descending order of values
sorted_ret_rec_dict = dict(sorted(retreived_recipes_dict.items(), key=lambda item: item[1], reverse=True))
sorted_ret_rec_dict

{'Paneer Butter Masala Recipe': 0.5,
 'Layered Paneer Butter Masala Biryani Recipe': 0.14586841826114608,
 'Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy': 0.13719695100067902,
 'Mixed Vegetable Handi Recipe': 0.13334985370681351,
 'Baby Corn Butter Masala Recipe': 0.1313520214671124,
 'Dry Navratan Korma Recipe': 0.13084232529109643,
 ' Paneer Makhani (Recipe In Hindi)': 0.1193631963132564,
 'Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy': 0.10742773519601138,
 'Mini Dal Samosa Curry / Sabzi Recipe': 0.10376934337053918,
 'Sweet and Spicy Pineapple Paneer Tikka Tawa Pizza Recipe': 0.09696278992882097,
 'Kaju Butter Masala Recipe': 0.09546479357180666,
 'Matar Paneer Kachori Korma Recipe - Green Peas And Paneer Kachori Curry': 0.09438026325362926,
 'Hyderabadi Shahi Mixed Vegetable Curry Recipe': 0.09429942275419602,
 'Shahi Bhindi Recipe in Cashew Nut Gravy - Okras in Cashew Nut Curry': 0.09249971550446508,
 'Reshmi Kofta Recipe in Rich Cashew Gravy': 0.09146333

In [27]:
import math
for recipe in sorted_ret_rec_dict:
    rating = get_recipe_rating(recipe)
    if math.isnan(rating):
        rating = 0
    new_value = (rating/5) * 0.2
    print(recipe, sorted_ret_rec_dict[recipe], rating ,new_value, new_value + sorted_ret_rec_dict[recipe]) 
    sorted_ret_rec_dict[recipe] = new_value + sorted_ret_rec_dict[recipe]
sorted_ret_rec_dict = dict(sorted(sorted_ret_rec_dict.items(), key=lambda item:item[1], reverse=True))
sorted_ret_rec_dict

Paneer Butter Masala Recipe 0.5 0 0.0 0.5
Layered Paneer Butter Masala Biryani Recipe 0.14586841826114608 4.9231511254019 0.19692604501607602 0.3427944632772221
Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy 0.13719695100067902 4.9288664525011 0.197154658100044 0.33435160910072304
Mixed Vegetable Handi Recipe 0.13334985370681351 4.787542662116 0.19150170648464004 0.32485156019145356
Baby Corn Butter Masala Recipe 0.1313520214671124 4.8670172802404 0.19468069120961604 0.3260327126767284
Dry Navratan Korma Recipe 0.13084232529109643 4.9078498293515 0.19631399317406 0.32715631846515647
 Paneer Makhani (Recipe In Hindi) 0.1193631963132564 0 0.0 0.1193631963132564
Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy 0.10742773519601138 4.8945929356621 0.195783717426484 0.3032114526224954
Mini Dal Samosa Curry / Sabzi Recipe 0.10376934337053918 0 0.0 0.10376934337053918
Sweet and Spicy Pineapple Paneer Tikka Tawa Pizza Recipe 0.09696278992882097 4.9096385542169 0.1963855421686

{'Paneer Butter Masala Recipe': 0.5,
 'Layered Paneer Butter Masala Biryani Recipe': 0.3427944632772221,
 'Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy': 0.33435160910072304,
 'Dry Navratan Korma Recipe': 0.32715631846515647,
 'Baby Corn Butter Masala Recipe': 0.3260327126767284,
 'Mixed Vegetable Handi Recipe': 0.32485156019145356,
 'Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy': 0.3032114526224954,
 'Sweet and Spicy Pineapple Paneer Tikka Tawa Pizza Recipe': 0.293348332097497,
 'Kaju Butter Masala Recipe': 0.29220071132721465,
 'Hyderabadi Shahi Mixed Vegetable Curry Recipe': 0.29053137541691604,
 'Shahi Bhindi Recipe in Cashew Nut Gravy - Okras in Cashew Nut Curry': 0.2887187237689291,
 'Dal Bukhara Recipe (Punjabi Style Black Urad Dal)': 0.2825575430688531,
 'Reshmi Kofta Recipe in Rich Cashew Gravy': 0.28230343861657414,
 'Jain Style Corn Palak Recipe': 0.28121170729129175,
 'Stuffed Capsicum Makhani Recipe': 0.27562168824305716,
 ' Paneer Makhani (Recipe 

In [28]:
for recipe in sorted_ret_rec_dict:
    nutrients = get_nutrients(recipe)
    n_score = balanced_diet(nutrients)
    # print(n_score)
    # print(sorted_ret_rec_dict[recipe])
    if n_score is None:
        n_score = 0.0
    final_score = n_score * 0.2 + sorted_ret_rec_dict[recipe]
    sorted_ret_rec_dict[recipe] = final_score

sorted_ret_rec_dict = dict(sorted(sorted_ret_rec_dict.items(), key = lambda item:item[1], reverse=True))
sorted_ret_rec_dict

{'Paneer Butter Masala Recipe': 0.5245740423098914,
 'Shahi Bhindi Recipe in Cashew Nut Gravy - Okras in Cashew Nut Curry': 0.42338853508968377,
 'Dry Navratan Korma Recipe': 0.36910994713967,
 'Layered Paneer Butter Masala Biryani Recipe': 0.3673685055871134,
 'Baby Corn Butter Masala Recipe': 0.3622076340603762,
 'Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy': 0.3587673958412458,
 'Sweet and Spicy Pineapple Paneer Tikka Tawa Pizza Recipe': 0.3527333775202783,
 'Mixed Vegetable Handi Recipe': 0.3489258158694828,
 'Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy': 0.3213899722161093,
 'Kaju Butter Masala Recipe': 0.316774753637106,
 'Dal Bukhara Recipe (Punjabi Style Black Urad Dal)': 0.3165437745833865,
 'Hyderabadi Shahi Mixed Vegetable Curry Recipe': 0.31534159919927673,
 'Stuffed Capsicum Makhani Recipe': 0.31360047195023993,
 'Reshmi Kofta Recipe in Rich Cashew Gravy': 0.307420038150176,
 'Jain Style Corn Palak Recipe': 0.2998698690980385,
 'Shahi Tindora Rec

In [29]:
for key in sorted_ret_rec_dict.keys():
     cook_time = get_cook_time_of_recipe(key)
     t_score = 0.1 * cook_time / 900

     sorted_ret_rec_dict[key] = sorted_ret_rec_dict[key] + t_score
sorted_ret_rec_dict = dict(sorted(sorted_ret_rec_dict.items(), key = lambda item:item[1], reverse=True))
sorted_ret_rec_dict


{'Paneer Butter Masala Recipe': 0.5290184867543358,
 'Shahi Bhindi Recipe in Cashew Nut Gravy - Okras in Cashew Nut Curry': 0.4278329795341282,
 'Dry Navratan Korma Recipe': 0.37133216936189223,
 'Layered Paneer Butter Masala Biryani Recipe': 0.3701462833648912,
 'Baby Corn Butter Masala Recipe': 0.36554096739370956,
 'Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy': 0.36210072917457914,
 'Sweet and Spicy Pineapple Paneer Tikka Tawa Pizza Recipe': 0.35662226640916717,
 'Mixed Vegetable Handi Recipe': 0.35170359364726056,
 'Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy': 0.3247233055494427,
 'Dal Bukhara Recipe (Punjabi Style Black Urad Dal)': 0.32321044125005316,
 'Hyderabadi Shahi Mixed Vegetable Curry Recipe': 0.31978604364372115,
 'Kaju Butter Masala Recipe': 0.31899697585932824,
 'Stuffed Capsicum Makhani Recipe': 0.3169338052835733,
 'Reshmi Kofta Recipe in Rich Cashew Gravy': 0.31408670481684264,
 'Jain Style Corn Palak Recipe': 0.30264764687581625,
 'Shahi 

In [33]:
recipes_to_be_removed = []
for recipe in sorted_ret_rec_dict:

    dish_specific_ingre = get_dish_specific_ingredients(recipe)
    count = 0
    for ingre in dish_specific_ingre:
        if ingre in ingredient_list:
            count += 1
    if count != len(dish_specific_ingre):
        recipes_to_be_removed.append(recipe)

for recipe in recipes_to_be_removed:
    sorted_ret_rec_dict.pop(recipe)
sorted_ret_rec_dict

{'Paneer Butter Masala Recipe': 0.5290184867543358,
 'Dry Navratan Korma Recipe': 0.37133216936189223,
 'Layered Paneer Butter Masala Biryani Recipe': 0.3701462833648912,
 'Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy': 0.36210072917457914,
 'Mixed Vegetable Handi Recipe': 0.35170359364726056,
 'Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy': 0.3247233055494427,
 'Hyderabadi Shahi Mixed Vegetable Curry Recipe': 0.31978604364372115,
 'Kaju Butter Masala Recipe': 0.31899697585932824,
 'Stuffed Capsicum Makhani Recipe': 0.3169338052835733,
 'Reshmi Kofta Recipe in Rich Cashew Gravy': 0.31408670481684264,
 'Jain Style Corn Palak Recipe': 0.30264764687581625,
 'Mini Dal Samosa Curry / Sabzi Recipe': 0.15057704709823644,
 'Aloo Matar Paneer Curry Recipe': 0.12790920491684332,
 ' Paneer Makhani (Recipe In Hindi)': 0.12214097409103418,
 'Mughlai Aloo Great Recipe - Mughlai Aloo Sabzi Recipe': 0.11149980147091955}

In [45]:
def RRS(ingredient_list, diet_type, course, diet):
    ingredient_list = stem_list(ingredient_list)
    retreived_recipes_list, retreived_recipes_dict  = get_recipes_with_ingredients_combinations_v3(ingredient_list, course, diet)
    
    #updating with rating score
    for recipe in retreived_recipes_dict:
        rating = get_recipe_rating(recipe)
        if math.isnan(rating):
            rating = 0
        new_value = (rating/5) * 0.2
        retreived_recipes_dict[recipe] = new_value + retreived_recipes_dict[recipe]

    #updating with nutrition score
    for recipe in retreived_recipes_dict:
        nutrients = get_nutrients(recipe)

        if diet_type == 3:
            n_score = balanced_diet(nutrients)
        elif diet_type == 2:
            n_score = weight_loss(nutrients)
        elif diet_type == 1:
            n_score = weight_gain(nutrients)
        else:
            n_score = None
        
        if n_score is None:
            n_score = 0.0
        final_score = n_score * 0.2 + retreived_recipes_dict[recipe]
        retreived_recipes_dict[recipe] = final_score

    #updating with time score
    for recipe in retreived_recipes_dict:
        cook_time = get_cook_time_of_recipe(recipe)
        t_score = 0.1 * cook_time / 900

        retreived_recipes_dict[recipe] = retreived_recipes_dict[recipe] + t_score

    #removing recipes which are not dish specific
    recipes_to_be_removed = []
    for recipe in retreived_recipes_dict:
            
            dish_specific_ingre = get_dish_specific_ingredients(recipe)
            count = 0
            for ingre in dish_specific_ingre:
                if ingre in ingredient_list:
                    count += 1
            if count != len(dish_specific_ingre):
                recipes_to_be_removed.append(recipe)

    for recipe in recipes_to_be_removed:
        retreived_recipes_dict.pop(recipe)

    #sorting the dictionary
    sorted_ret_rec_dict = dict(sorted(retreived_recipes_dict.items(), key = lambda item:item[1], reverse=True))
    #top 5 recipes
    Recipes_dict = dict.fromkeys(list(sorted_ret_rec_dict.keys())[:5], [])
    for recipe in Recipes_dict:
        value_list = []
        value_list.append(get_recipe_url(recipe))
        served_with_recipes = get_served_with(recipe)
        top_5_recipes = get_top_5_recipes(served_with_recipes)
        value_list.append(top_5_recipes)
        Recipes_dict[recipe] = value_list

    return Recipes_dict
        


***************

# RESULT

In [49]:
ingredient_list1 = ['garam masala powder', 'paneer','fresh cream', 'butter', 'cashew nut', 'cumin powder', 'kasuri methi']
#diet_type : 1 for weight gain, 2 for weight loss, 3 for balanced diet
diet_type = 3
#course : Main Course, Dessert, Appetizer, Side Dish, Breakfast, Snack
course = "Main Course"
#diet : Vegetarian, Non-Vegetarian
diet = "Vegetarian"

result = RRS(ingredient_list1, diet_type, course, diet)
i = 0
for recipe in result:
    print(i+1, ":", recipe, result[recipe][0])
    print("Can be served with: ")
    for r in result[recipe][1]:
        print(r)
    i += 1


1 : Paneer Butter Masala Recipe https://www.archanaskitchen.com/paneer-butter-masala-recipe
Can be served with: 
Nawabi Tawa Paratha Recipe
Tawa Paratha Recipe - Plain Paratha
Butter Garlic Naan Recipe - Garlic Flavoured Flatbread
Whole Wheat Methi Garlic Naan Recipe  | Oven Method with yeast
Jowar Garlic Naan Recipe- Healthy Sorghum Naan
2 : Veg Makhanwala Recipe - North Indian Mixed Vegetables Gravy http://www.archanaskitchen.com/veg-makhanwala-recipe-north-indian-mixed-vegetables-gravy
Can be served with: 
Malai Matar Paneer Recipe (Cottage Cheese & Peas In Rich Creamy Gravy)
Vegan Baingan Makhani Recipe (Eggplant In Rich Coconut Gravy)
Kerala Style Rich Creamy Egg Cashew Curry Recipe
Baingan Musallam Recipe (Mughlai Style Eggplant Simmered In Rich Tomato Curry)
Protein Rich Paneer And Horse Gram Soup Recipe - Kollu/ Kulith Soup
3 : Dry Navratan Korma Recipe http://www.archanaskitchen.com/dry-navratan-korma-recipe
Can be served with: 
 Tomato Onion Cucumber Raita (Recipe In Hindi)
K